In [11]:
# in reference to:
# https://towardsdatascience.com/scraping-reddit-with-praw-76efc1d1e1d9
# https://praw.readthedocs.io/en/latest/code_overview/praw_models.html
# https://www.reddit.com/wiki/search

from os.path import isfile
import praw
import pandas as pd
from time import sleep


In [33]:
from os.path import isfile
import praw
import pandas as pd
from time import sleep

# Get credentials from DEFAULT instance in praw.ini
# Use your own tokens
reddit = praw.Reddit(
    client_id="tpkjzcHMAynTvw",
    client_secret="WdtCdrgnWMfZphmCgkl_fVFH-uA",
    user_agent="scraper",
    username="Whatchamacalit1",
    password="Tango9802!"
)


class SubredditScraper:

    def __init__(self, query, sort='new', lim=900, mode='w'):
        self.query = query
        self.sort = sort
        self.lim = lim
        self.mode = mode
        
        if not isinstance(query, str):
            try:
                self.query = self.query.replace(" ", "%20")
            except AttributeError:
                raise

        print(
            f'SubredditScraper instance created with values '
            f'query = {query}, sort = {sort}, lim = {lim}, mode = {mode}')

    def set_sort(self):
        if self.sort == 'new':
            return self.sort, reddit.subreddit('all').search(self.query, sort='new', time_filter='month')
        elif self.sort == 'top':
            return self.sort, reddit.subreddit('all').search(self.query, sort='top', time_filter='month')
        elif self.sort == 'hot':
            return self.sort, reddit.subreddit('all').search(self.query, sort='hot', time_filter='month')
        else:
            self.sort = 'relevance'
            print('Sort method was not recognized, defaulting to relevance.')
            return self.sort, reddit.subreddit('all').search(self.query, time_filter='month')

    def get_posts(self, printer=False):

        sub_dict = {
            'game_title': [], 'selftext': [], 'title': [], 'id': [], 'sorted_by': [],
            'num_comments': [], 'score': [], 'ups': [], 'downs': []}
        csv = f'game_titles.csv'

        # Attempt to specify a sorting method.
        sort, subreddit = self.set_sort()

        # Set csv_loaded to True if csv exists since you can't evaluate the
        # truth value of a DataFrame.
        df, csv_loaded = (pd.read_csv(csv), 1) if isfile(csv) else ('', 0)

        print(f'csv = {csv}')
        print(f'After set_sort(), sort = {sort} and query = {self.query}')
        print(f'csv_loaded = {csv_loaded}')

        print(f'Collecting information from r/all for search term "{self.query}".')
        for post in subreddit:

            # Check if post.id is in df and set to True if df is empty.
            # This way new posts are still added to dictionary when df = ''
            unique_id = post.id not in tuple(df.id) if csv_loaded else True

            # Save any unique, non-stickied posts with descriptions to sub_dict.
            if unique_id:
                sub_dict['game_title'].append(self.query)
                sub_dict['selftext'].append(post.selftext)
                sub_dict['title'].append(post.title)
                sub_dict['id'].append(post.id)
                sub_dict['sorted_by'].append(sort)
                sub_dict['num_comments'].append(post.num_comments)
                sub_dict['score'].append(post.score)
                sub_dict['ups'].append(post.ups)
                sub_dict['downs'].append(post.downs)
            sleep(0.1)

        # pprint(sub_dict)
        new_df = pd.DataFrame(sub_dict)
        
        # decide whether or no to print
        if printer is True:
            print(
                f'{len(new_df)} posts were collected but they were not '
                f'added to {csv} because mode was set to "{self.mode}"')
            return

        # Add new_df to df if df exists then save it to a csv.
        if 'DataFrame' in str(type(df)) and self.mode == 'w':
            pd.concat([df, new_df], axis=0, sort=0).to_csv(csv, index=False)
            print(
                f'{len(new_df)} new posts collected and added to {csv}')
        elif self.mode == 'w':
            new_df.to_csv(csv, index=False)
            print(f'{len(new_df)} posts collected and saved to {csv}')
        else:
            print(
                f'{len(new_df)} posts were collected but they were not '
                f'added to {csv} because mode was set to "{self.mode}"')



In [13]:
if __name__ == '__main__':
    #SubredditScraper('statistics', lim=997, mode='w', sort='new').get_posts()
    SubredditScraper('Over Watch', lim=997, sort='new').get_posts()

SubredditScraper instance created with values query = Over Watch, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Over Watch
csv_loaded = 1
0 new posts collected and added to game_titles.csv


In [35]:
import unittest
import os

class TestRedditMethods(unittest.TestCase):

    def test_bad_input(self):
        self.assertRaises(AttributeError, SubredditScraper, query=123, lim=997, sort='new')
        
    def test_good_input(self):
        SubredditScraper('Over Watch', lim=997, sort='new').get_posts()
        self.assertTrue(os.path.isfile('./game_titles.csv'))
 

In [36]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.

SubredditScraper instance created with values query = Over Watch, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Over Watch
csv_loaded = 1


.

3 new posts collected and added to game_titles.csv



----------------------------------------------------------------------
Ran 2 tests in 12.763s

OK
